In [1]:
from __future__ import division
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/'My Drive'

/content/gdrive/My Drive


In [0]:
!rm -rf baseball_lab

In [5]:
!git clone https://github.com/physhik/baseball_lab.git

Cloning into 'baseball_lab'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 141 (delta 68), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (141/141), 1.60 MiB | 6.93 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [5]:
cd baseball_lab/src

/content/gdrive/My Drive/baseball_lab/src



Other park correctors in baseball-reference considers the correction of the initial factor 

Initial factor of a park is computed by (Scored RUNs of home team at home per home game number + Allowed RUNs of home team at home per home game number )/ (SR of HT at away per AGN + AR of HT at away per AGN ). In other words, it presents how easy to make and allow runs at home.

The rough park factor can be calculated by (initial factor -1)/2 + 1. For example, if initial factor is 1.2, the rough park factor not considering small corrections is 1.1.

I recommend you to read the [this link](https://www.baseball-reference.com/about/parkadjust.shtml) before further discussion. The above link introduce how to compute the park factor for baseball reference. 

I especially focused on the step 3, OPC (other park corrector).

"Make corrections for the fact that the other road parks' total difference from the league average is offset by the park rating of the club that is being rated"

BTW, it assumes the initial factors of the other parks are all identical, or have the same number of games with N-1 teams (N=30).

More details with formula will be presented at [my personal blog](http://physhik.com/opc/).

However, it's slighty off because a mlb team have more games in the same division. This was already pointed out in other posts such as [this](https://www.reddit.com/r/Sabermetrics/comments/4kjlw9/are_park_factors_real/) 

I calculated the more precise OPC x Initial factor from the number of all the games of all the teams by web crawling mlb schedule.

I also updated for baseball-reference 'Pitching/batting park factor(PPF/BPF)' via 'Team pitching/batting rating(TPR/TBR)'. The formula to this also seems to have room to be improved to me, and still on progressing. 

The park factor in baseball reference and fangraphs are scaled separatedly for NL/AL. I did not split the league, and DH/non-DH effect was not considered yet. Thus, the resultant park factor I calucuated overestimates AL yet. I will modify this soon. 

In [0]:
from parkfactor import *

Now we need the records of GP (games played), W at home and away for each team and park. 


You need to run gamestadium function, but it takes much time. If you are interested in 2014 ~ 2018, I saved the dictionary in /downloads directory, and you can load the data. 

In [0]:
# Use the saved data 

import json 
year = 2018

with open('../../baseball/teamd'+str(year)+'.json') as json_file:
    teamd = json.load(json_file)

In [0]:
with open('teamd2018.json', 'w') as outfile:
    json.dump(teamd, outfile)

In [0]:
# make your own 

year = # Put your own concerend season 

teamd = {}
teaml = ['lad','col','sf','ari','sd','stl','pit','cin','chc','mil','phi','nym','wsh','mia','atl','hou','tex','oak','sea','laa','min','kc','cle','chw','det','nyy', 'tor','bos','tb','bal'] 

for t in teaml:
  teamd[t] = {'Home' :{}, 'Away':{}}

teamd = gamestadium(year, teaml, teamd)


In [11]:
pf = oldBP(2018,teamd,True)

pf

The determinant of 30x30 matrice must be near zero,  -0.0068371304463738
Ideally sum of the factors is 30, and here is  30.091585223126533


,2018,2018IPC,OPCxRUN/IPC,mOPCxRUN/IPC,TBR,TPR,BPF,PPF
Arizona,1.056923,1.001373,1.053475,1.057288,0.931220,0.867717,1.032365,1.030099
Atlanta,1.119760,1.002756,1.112243,1.060608,1.019139,0.884758,1.033366,1.028580
Baltimore,0.981675,0.993917,0.988287,0.996332,0.871711,1.234211,0.990232,1.002665
Boston,1.077763,0.995855,1.079451,1.078681,1.164462,0.871285,1.042612,1.032131
Chicago Cubs,1.078942,0.995615,1.080850,1.046355,1.022224,0.869186,1.027011,1.021595
Chicago White Sox,0.943152,1.001364,0.943656,0.958064,0.935387,1.197914,0.973114,0.981943
Cincinnati,1.127809,0.995233,1.128403,1.106399,0.918454,1.079618,1.048487,1.054330
Cleveland,1.127721,0.995194,1.128364,1.134220,1.059103,0.848031,1.070405,1.062630
Colorado,1.271422,0.997569,1.263092,1.269382,0.946988,0.911695,1.133498,1.132116
Detroit,0.945430,0.991851,0.954935,0.969351,0.891724,1.115587,0.981293,0.988896
